In [ ]:
pip install tensorflow opencv-python Pillow


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

# Load the low-quality CCTV image (Replace with your own dataset path)
img_path = 'path_to_your_low_quality_cctv_image.jpg'
low_quality_image = cv2.imread(img_path)

# Resize for input to the model if necessary
low_quality_image = cv2.resize(low_quality_image, (128, 128))


In [ ]:
#Super-Resolution Using SRCNN (Super-Resolution Convolutional Neural Network)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D

# Define a simple SRCNN model for super-resolution
def build_srcnn_model():
    model = Sequential()

    # First Convolutional Layer
    model.add(Conv2D(64, (9, 9), activation='relu', padding='same', input_shape=(None, None, 3)))

    # Second Convolutional Layer
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))

    # Third Convolutional Layer
    model.add(Conv2D(3, (5, 5), padding='same'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Initialize and compile the model
srcnn_model = build_srcnn_model()

# Train the model on the low-res/high-res dataset (Here we would use the provided dataset)
# srcnn_model.fit(x_train_low_res, y_train_high_res, epochs=10, batch_size=16)


In [ ]:
Step 4: Image Denoising and Deblurring
# Function to denoise the image using a Gaussian filter
def denoise_image(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

# Function to deblur the image using Wiener filter
def deblur_image(image):
    deblurred = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return deblurred

# Apply denoising and deblurring
denoised_image = denoise_image(low_quality_image)
deblurred_image = deblur_image(denoised_image)

# Save the processed images
cv2.imwrite('denoised_image.jpg', denoised_image)
cv2.imwrite('deblurred_image.jpg', deblurred_image)


In [ ]:
Step 5: Model for Face Reconstruction (Using Autoencoders or GANs)
from tensorflow.keras.layers import Input, UpSampling2D, Conv2DTranspose
from tensorflow.keras.models import Model

# Define a simple autoencoder for image reconstruction
def build_autoencoder():
    input_img = Input(shape=(128, 128, 3))

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

    # Decoder
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

# Build and compile the autoencoder
autoencoder = build_autoencoder()

# Train the autoencoder on low-res / high-res dataset pairs
# autoencoder.fit(x_train_low_res, y_train_high_res, epochs=20, batch_size=16)

# Test the autoencoder with a low-quality image
reconstructed_image = autoencoder.predict(np.expand_dims(low_quality_image, axis=0))

# Save the reconstructed image
cv2.imwrite('reconstructed_image.jpg', reconstructed_image[0] * 255)  # Convert to image scale


In [ ]:
#Step 6: Comparing the Original and Reconstructed Image
import matplotlib.pyplot as plt

# Load original and reconstructed images for comparison
original_image = cv2.imread('path_to_original_cctv_image.jpg')
reconstructed_image = cv2.imread('reconstructed_image.jpg')

# Show original vs reconstructed
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(reconstructed_image, cv2.COLOR_BGR2RGB))
plt.title('Reconstructed Image')

plt.show()


In [ ]:
#Step 7: Real-Time Processing Capability (Optional)
# Capture video from CCTV feed
cap = cv2.VideoCapture('path_to_video_feed')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process each frame (denoise, deblur, reconstruct)
    frame_denoised = denoise_image(frame)
    frame_deblurred = deblur_image(frame_denoised)
    reconstructed_frame = autoencoder.predict(np.expand_dims(frame_deblurred, axis=0))[0]

    # Display the processed frame
    cv2.imshow('Reconstructed Frame', reconstructed_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
